In [1]:
import pandas as pd
import random

https://gengo.ai/datasets/15-free-sentiment-analysis-datasets-for-machine-learning/

In [2]:
import json
i = 0
reviews_lst_good= []
reviews_lst_bad = []


with open('C:\\Users\\denis.pekhterev\\Downloads\\Experiment_old\\Books_5.json', 'r') as f:
        for line in f:
            if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                reviews_lst_good.append (json.loads(line))
            
            elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                reviews_lst_bad.append (json.loads(line))

            if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                break
                
random.shuffle(reviews_lst_good)
random.shuffle(reviews_lst_bad)

In [3]:
reviewerID_lst = []
reviewerName_lst = []
reviewText_lst = []
overall_lst = []
summary_lst = []

# reviews_lst [0]['overall']<5 #['asin']for 

for i in reviews_lst_good:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
for i in reviews_lst_bad:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
df = pd.DataFrame ()

df['reviewerID'] = reviewerID_lst
# df['reviewerName'] = reviewerName_lst
df['reviewText'] = reviewText_lst
df['overall'] = overall_lst
df['summary'] = summary_lst

In [4]:
df ['mark'] = 0
df.loc[df['overall'] > 3,'mark'] = 1

print (len(df[df['mark'] > 0]))

500001


In [5]:
df[(df['mark'] != 0)&(df['mark'] != 1)]

,reviewerID,reviewText,overall,summary,mark


In [6]:
reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])

In [7]:
# list(df[df['overall'] > 3]['reviewText'][:2])
reviews_train [:2]

["Insurgent, book 2 in the Divergent series is another of my most anticipated reads for this year and it held me captivated from the very first page.I honestly don't know where to start! Divergent for me was all sorts of amazing; it was everything I love in a dystopian novel and although I really enjoyed Insurgent, the more morose feel seemed to bring it down a level for me.Insurgent takes us through the aftermath of Divergent, lives have been lost, loyalties have been divided and the factions are more or less at war. Tris Prior has a lot of guilt on her shoulders, she is suffering terribly with what she has been through and what she has lost; her strength and determination were still present but along with it came the negativity and her grief, she was dealing with a lot of emotion and it certainly came out through her sometimes reckless and selfish actions.Four/Tobias makes a few revelations of his own, I absolutely adore him as a character; he is strong, protective and determined and

In [8]:
import re
from nltk.corpus import stopwords 

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [10]:

y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

# for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
#     lr = LogisticRegression(C=c)
#     lr.fit(X_train, y_train)
#     print ("Accuracy for C=%s: %s" 
#            % (c, accuracy_score(y_val, lr.predict(X_val))))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
final_model = LogisticRegression(C=0.5)
final_model.fit(X,  y)
print ("Final Accuracy: %s" 
       % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                        , final_model.predict(X_test)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy: 0.91515


In [12]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

('klausner', 4.200585683240843)
('ariely', 2.8356831815044803)
('everneath', 2.6124595643870463)
('maxon', 2.548042693956325)
('spong', 2.503794565750101)
('booksneeze', -3.4660027102033775)
('waterbrook', -2.7733974365937835)
('killgore', -2.7173856084710564)
('scarpetta', -2.6039862116577344)
('booksneezecom', -2.5929997283216117)


In [13]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:120]:
    print (best_positive)

writers = ['klausner','rollins','ehrman','gaiman','spong','olaf','steinbeck','harpercollins','discworld',
'kingsolver','pratchett','maxon']

('klausner', 4.200585683240843)
('ariely', 2.8356831815044803)
('everneath', 2.6124595643870463)
('maxon', 2.548042693956325)
('spong', 2.503794565750101)
('olaf', 2.335943150736356)
('ehrman', 2.2755094891117817)
('pleasantly', 2.2397345696720072)
('aria', 2.227326464942671)
('margolin', 2.209052535420335)
('erdrich', 2.016718935236432)
('wynne', 2.001600642188314)
('proses', 1.9956662863936512)
('tamani', 1.9886643508909623)
('kaiden', 1.9801171303234608)
('hobb', 1.9683015005540692)
('maclean', 1.952409460033418)
('zinsser', 1.94943085254741)
('lobdell', 1.9294905306995673)
('argeneau', 1.925562957231128)
('halpern', 1.915425482499514)
('wallis', 1.9136862005436155)
('rollins', 1.9082115838869715)
('humbert', 1.9015820908296284)
('refreshing', 1.9004044314727193)
('gaiman', 1.8980815479534694)
('quibble', 1.8979643410512592)
('harpercollins', 1.8927034055101044)
('alafair', 1.8753738127073951)
('dany', 1.853341642609548)
('gitomer', 1.8495983741606035)
('septimus', 1.842669829383629

# Analyse reviews

In [15]:
import json
import random
import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []


    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                    break
    
    random.shuffle(reviews_lst_good)
    random.shuffle(reviews_lst_bad)
                    
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []

    # reviews_lst [0]['overall']<5 #['asin']for 

    for i in reviews_lst_good:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    for i in reviews_lst_bad:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    df = pd.DataFrame ()

    df['reviewerID'] = reviewerID_lst
    # df['reviewerName'] = reviewerName_lst
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
    df['summary'] = summary_lst
    
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    
    
    reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
    reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])



    reviews_train_clean = preprocess_reviews(reviews_train)
    reviews_test_clean = preprocess_reviews(reviews_test)
    
    cv = CountVectorizer(binary=True)
    cv.fit(reviews_train_clean)
    X = cv.transform(reviews_train_clean)
    X_test = cv.transform(reviews_test_clean)
    
    y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size = 0.75
    )
    
    print ('Подбор параметра для: ', filename)
    
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, lr.predict(X_val))))
    
    
    final_model = LogisticRegression(C=parametr)
    final_model.fit(X,  y)
    print ("Final Accuracy: %s" 
           % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                            , final_model.predict(X_test)))
    
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:5]:
        print (best_positive)
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:5]:
        print (best_negative)
        
    del reviews_lst_good
    del reviews_lst_bad

In [ ]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.5)
        gc.collect()
        

Health_and_Personal_Care_5.json

Подбор параметра для:  G:\Мой диск\Work\Experiment\Health_and_Personal_Care_5.json
Accuracy for C=0.01: 0.9375511763742562
Accuracy for C=0.05: 0.94242316720345
Accuracy for C=0.25: 0.9450161034990993
Accuracy for C=0.5: 0.9445930454719144
Accuracy for C=1: 0.9438561056826246
Final Accuracy: 0.73015
('deducted', 1.7656824591106735)
('pleasantly', 1.656820137862464)
('pleased', 1.6369400592065257)
('skeptical', 1.5956846843153587)
('highly', 1.5722485210054)
('disappointing', -2.5610881508681285)
('worthless', -2.4379621815169608)
('disappointment', -2.143615023529117)
('unacceptable', -2.0601684486255207)
('poorly', -1.905376850288279)
Movies_and_TV_5.json



In [ ]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.25)
        gc.collect()
        

In [ ]:
import json
import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []


    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=500000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=500000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 1000000:
                    break
    reviews_lst_good = random.shuffle(reviews_lst_good)
    reviews_lst_bad = random.shuffle(reviews_lst_bad)  
    
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []

    # reviews_lst [0]['overall']<5 #['asin']for 

    for i in reviews_lst_good:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    for i in reviews_lst_bad:
        reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        summary_lst.append (i['summary'])

    df = pd.DataFrame ()

    df['reviewerID'] = reviewerID_lst
    # df['reviewerName'] = reviewerName_lst
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
    df['summary'] = summary_lst
    
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    reviews_train = list(df[df['overall'] > 3]['reviewText'][:-10000]) + list(df[df['overall'] < 3]['reviewText'][:-10000])
    reviews_test = list(df[df['overall'] > 3]['reviewText'][-10000:]) + list(df[df['overall'] < 3]['reviewText'][-10000:])



    reviews_train_clean = preprocess_reviews(reviews_train)
    reviews_test_clean = preprocess_reviews(reviews_test)
    
    cv = TfidfVectorizer(binary=True)
    cv.fit(reviews_train_clean)
    X = cv.transform(reviews_train_clean)
    X_test = cv.transform(reviews_test_clean)
    
    y = list(df[df['overall'] > 3]['mark'][:-10000]) + list(df[df['overall'] < 3]['mark'][:-10000])
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size = 0.75
    )
    
    print ('Подбор параметра для: ', filename)
    print ('    ')
    
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, lr.predict(X_val))))
    
    
    final_model = LogisticRegression(C=parametr)
    final_model.fit(X,  y)
    print ("Final Accuracy: %s" 
           % accuracy_score( list(df[df['overall'] > 3]['mark'][-10000:]) + list(df[df['overall'] < 3]['mark'][-10000:])
                            , final_model.predict(X_test)))
    
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:5]:
        print (best_positive)
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:5]:
        print (best_negative)
        
    del reviews_lst_good
    del reviews_lst_bad

In [ ]:
import os
import gc

os.listdir('G:\\Мой диск\\Work\\Experiment')

for i in os.listdir('G:\\Мой диск\\Work\\Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints':
        print (i)
        print ('   ')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.25)
        gc.collect()
        print ('   ')
        classify_it ('G:\\Мой диск\\Work\\Experiment\\' + i, 0.5)
        

In [ ]:
import random

list = [20, 16, 10, 5];
random.shuffle(list)
print ("Reshuffled list : ",  list)

In [ ]:
# BackUP

In [ ]:
# df = pd.read_json('Books_5.json')
# import ijson
# filename = "Books_5.json"
# with open(filename, 'r') as f:
#     objects = ijson.items(f, 'meta.view.columns.item')
#     columns = list(objects)
#     print (objects)
# handle = open("Books_5.json", "r")

# while True:
#     data = handle.read(20000)
#     if not data:
#         break